In [103]:
import pandas as pd

reviews = pd.read_json('data/ys-reviews-with-categories.json')

restuarants = reviews.loc[reviews.category == "restaurant"]

sample: list[str] = restuarants.text.tolist()

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from string import punctuation

import yake as yk

stop_words = stopwords.words("english")
keyword_extractor = yk.KeywordExtractor(n=3,top=10000)
filtered_words = []

# def key_ngram_per_sentence (review_text) -> list[str]:
#     sentences = sent_tokenize(review_text)
#     key_ngrams = []
#     for sentence in sentences:
#         keywords = keyword_extractor.extract_keywords(removed_stopwords(sentence))
#         if len(keywords) > 0: key_ngrams.append(keywords[0][0])
#     return key_ngrams    
    
# def key_ngram (review_text) -> list[str]:
#     key_ngrams = []
#     keywords = keyword_extractor.extract_keywords(removed_stopwords(review_text))
#     if len(keywords) > 0: key_ngrams.append(keywords[0][0])
#     return key_ngrams    
    
def removed_stopwords(text: str) -> str:
    # make lower case and remove all punctuations
    text = text.lower().translate(str.maketrans('', '', punctuation))
    tokens = word_tokenize(text)
    # filter stopwords    
    filtered = [word for word in tokens if word not in stop_words]
    filtered = [word for word in tokens if word not in filtered_words]
    return ' '.join(filtered)


corpus = "\n".join(sample)
corpus = removed_stopwords(corpus)

extractions = keyword_extractor.extract_keywords(corpus)

with open('keywords.csv', 'w') as outfile:
    outfile.write(pd.DataFrame(extractions).to_csv(index=False))

# keywords = list(set([extraction[0] for extraction in extractions]))

# keywords

In [ ]:
keywords_df = pd.read_csv('keywords.csv')
keywords = keywords_df[0:1000][keywords_df.columns[0]].tolist()
keywords

['food service food',
 'food food service',
 'service food food',
 'service food service',
 'food customer service',
 'friendly staff food',
 'friendly service food',
 'fast service food',
 'food fast service',
 'food friendly service',
 'food service service',
 'food excellent service',
 'nice place food',
 'customer service food',
 'food friendly staff',
 'place service food',
 'place food service',
 'food excellent food',
 'service food nice',
 'service excellent food',
 'love love love',
 'food nice place',
 'excellent food food',
 'food delicious food',
 'service friendly staff',
 'service delicious food',
 'service awesome food',
 'service service food',
 'excellent food service',
 'food service',
 'service food love',
 'food service love',
 'food service prices',
 'restaurant food service',
 'food place service',
 'food food',
 'service food',
 'place food place',
 'food nice service',
 'excellent service food',
 'delicious food service',
 'food service fast',
 'food food friend

In [ ]:
[ngram for ngram in keywords if 'food' not in ngram.split(' ')]

In [132]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering

# Sample trigrams
trigrams = keywords

# Convert trigrams to TF-IDF vectors
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(trigrams)

# Calculate cosine similarity matrix
cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Perform hierarchical clustering
n_clusters = 50  # You can adjust the number of clusters
clustering = AgglomerativeClustering(n_clusters=n_clusters, metric='cosine', linkage='average')
cluster_labels = clustering.fit_predict(cosine_sim_matrix)

# Assign trigrams to clusters
trigram_clusters = {}
for trigram, label in zip(trigrams, cluster_labels):
    if label not in trigram_clusters:
        trigram_clusters[label] = []
    trigram_clusters[label].append(trigram)

# Print the trigram clusters
for label, cluster in trigram_clusters.items():
    if len(cluster) > 1 : print(f'Cluster {label + 1}: {cluster}')


Cluster 10: ['food service food', 'food food service', 'service food food', 'service food service', 'food service service', 'service service food', 'food service', 'service food', 'food quick service', 'service tasty food', 'meal food service', 'yummy food service', 'tasty food service', 'service bad service', 'quick service food', 'service breakfast food', 'slow service food', 'food slow service', 'eat service food', 'service food taco', 'service food ordered', 'food service reasonable', 'food bad service', 'service bad food', 'bad service food', 'service food long', 'food service coffee', 'lunch food service', 'bar food service', 'food service translated', 'service food beer', 'food service beer', 'food beer service', 'service food tastes', 'cheese food service', 'food service family', 'eat service service', 'portions food service', 'breakfast service food', 'italian food service', 'food selection service', 'service wonderful food', 'wonderful food service', 'food service highly', 's